In [1]:
import pandas as pd
import regex as re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import string
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import nltk
from nltk import PorterStemmer , word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
nltk.download("wordnet")
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path= "/content/drive/MyDrive/DataSets/spam.csv"
# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path, encoding='latin1', usecols=['v1', 'v2'])

# Display the first few rows
print(df.head())


     v1                                                 v2
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [4]:
# changing labels

df.columns =['label', 'email']
# Assuming 'ham' is encoded as 0 and 'spam' is encoded as 1
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

df.head()

,label,email
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
def preprocess(Email):
    # Remove punctuation
    Email = re.sub(r'[^\w\s]', '', Email)

    # Convert to lowercase
    Email = Email.lower()

    # Tokenize words and remove stopwords
    stop_words = set(stopwords.words("english"))
    Stemmer = PorterStemmer()
    words = word_tokenize(Email)
    filtered_words = [Stemmer.stem(word) for word in words if word not in stop_words]
    preprocessed_email = " ".join(filtered_words)

    return preprocessed_email

In [6]:
def embed_text(data, embed_type='bow'):
    if embed_type == 'bow':
        vectorizer = CountVectorizer()
    elif embed_type == 'tfidf':
        vectorizer = TfidfVectorizer()
    else:
        raise ValueError("Invalid 'embed_type'. Use 'bow' or 'tfidf'.")

    data = vectorizer.fit_transform(data)

    return data

In [7]:
#Getting datasets using bow and embed

df['processed_email'] = df['email'].apply(preprocess)


In [9]:
data_bow = embed_text(df['processed_email'], embed_type='bow')
data_tfidf = embed_text(df['processed_email'], embed_type='tfidf')

In [10]:
X_bow = data_bow
X_tfidf= data_tfidf
Y = df['label']
#spliting data
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bow, Y, test_size=0.2, random_state=42)

X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, Y, test_size=0.2, random_state=42)

In [11]:
# Create and train the regression model on bow dataset
regression_model = LogisticRegression()
regression_model.fit(X_train_bow, y_train_bow)

# Make predictions on the test set for regression
predictions_regression_bow = regression_model.predict(X_test_bow)

accuracy_bow_reg = accuracy_score(y_test_bow, predictions_regression_bow)
print("Accuracy :", accuracy_bow_reg)

Accuracy : 0.97847533632287


In [12]:
# Create  the regression model on tfidf dataset
regression_model.fit(X_train_tfidf, y_train_tfidf)

# Make predictions on the test set for regression
predictions_regression_tfidf = regression_model.predict(X_test_tfidf)

accuracy_tfidf_reg = accuracy_score(y_test_tfidf, predictions_regression_tfidf)
print("Accuracy :", accuracy_tfidf_reg)

Accuracy : 0.9434977578475336


In [13]:
# Create the random forest on bow dataset
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_bow, y_train_bow)

# Make predictions on the test set for regression
predictions_rf_bow = regression_model.predict(X_test_bow)

accuracy_bow_rf = accuracy_score(y_test_bow, predictions_rf_bow)
print("Accuracy :", accuracy_bow_rf)

Accuracy : 0.968609865470852


In [14]:
# Create  the regression model on tfidf dataset
rf_classifier.fit(X_train_tfidf, y_train_tfidf)

# Make predictions on the test set for regression
predictions_rf_tfidf = rf_classifier.predict(X_test_tfidf)

accuracy_tfidf_rf = accuracy_score(y_test_tfidf, predictions_rf_tfidf)
print("Accuracy :", accuracy_tfidf_rf)

Accuracy : 0.9766816143497757
